In [ ]:
import numpy as np
import torch
from sklearn.model_selection import train_test_split

# Charger les segments
data = np.load("segments_npz/segments_batch_1.npz")
X = data['X']  # shape: (nb_segments, 256)
y = data['y']  # shape: (nb_segments,)

# Normalisation (important pour LSTM)
X = (X - X.mean()) / X.std()

# Découper en train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convertir en tenseurs pour LSTM (batch, sequence, features)
X_train = torch.tensor(X_train, dtype=torch.float32).unsqueeze(-1)  # (batch, 256, 1)
X_test = torch.tensor(X_test, dtype=torch.float32).unsqueeze(-1)
y_train = torch.tensor(y_train, dtype=torch.long)
y_test = torch.tensor(y_test, dtype=torch.long)


In [ ]:
import torch.nn as nn

class EEG_LSTM(nn.Module):
    def __init__(self):
        super(EEG_LSTM, self).__init__()
        self.lstm = nn.LSTM(input_size=1, hidden_size=64, num_layers=1, batch_first=True)
        self.fc = nn.Linear(64, 2)  # 2 classes : crise ou non

    def forward(self, x):
        out, _ = self.lstm(x)       # out: (batch, seq_len, hidden)
        out = out[:, -1, :]         # on prend le dernier état caché
        return self.fc(out)


In [ ]:
from torch.utils.data import TensorDataset, DataLoader
import torch.optim as optim

# Préparer le DataLoader
batch_size = 64
train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=batch_size, shuffle=True)
test_loader = DataLoader(TensorDataset(X_test, y_test), batch_size=batch_size)

# Initialiser modèle, perte, optimiseur
model = EEG_LSTM()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Entraînement
for epoch in range(10):
    model.train()
    total_loss = 0
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"Époque {epoch+1} - Perte: {total_loss / len(train_loader):.4f}")


In [ ]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for batch_X, batch_y in test_loader:
        outputs = model(batch_X)
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == batch_y).sum().item()
        total += batch_y.size(0)

print(f"Précision sur test : {100 * correct / total:.2f}%")
